In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from lib.ECP import Agent, ErgodicCP

In [ ]:
kappa_true = 10
kappa_est = np.random.uniform(1, 100)

lambda_buy = 0.4
lambda_sell = 0.4
q_upper, q_lower = 30, -30
phi = 1e-6

simga = 0.01

T=50
dt=1
K_upper=100
delta0=1e-8

In [ ]:
ErgodicCP(
    lambda_buy=lambda_buy,
    lambda_sell=lambda_sell,
    q_upper=q_upper,
    q_lower=q_lower,
    phi=phi,
    kappa=kappa_true,
).plot_EControl

In [ ]:
Regret_MC = 250
regret_samples = 5


MC_regret = []
MC_kappa = []


for _ in range(Regret_MC):
    regret_list = []
    kappa_list = []
    for _ in range(regret_samples):
        kappa_est = np.random.uniform(50, 100)

        agent = Agent(
            lambda_buy=lambda_buy,
            lambda_sell=lambda_sell,
            q_upper=q_upper,
            q_lower=q_lower,
            phi=phi,
            kappa_est=kappa_est,
            T=T,
            dt=dt,
            K_upper=K_upper,
            delta0=delta0, 
        )

        agent.learning(sigma=simga, kappa_true=kappa_true)

        kappa_list.append(agent.kappa_learnlist)
        regret_list.append(agent.regret(kappa_true=kappa_true))

    kappa_list = np.array(kappa_list)
    regret_list = np.array(regret_list)
    MC_regret.append(np.mean(regret_list, axis=0))
    MC_kappa.append(np.mean(kappa_list, axis=0))

MC_kappa = np.array(MC_kappa)
MC_regret = np.array(MC_regret)

In [ ]:
std = np.std(MC_kappa-kappa_true, axis=0)
ci = 1.64 * std/np.sqrt(len(agent.ts))
y = np.mean(MC_kappa, axis=0)-kappa_true

fig, ax = plt.subplots()
ax.loglog(agent.ts, y)
ax.fill_between(agent.ts, (y-ci), (y+ci), color='b', alpha=.1)

# plt.loglog(agent.ts[1:], agent.ts[1:]**(-0.5), '--')
plt.xlabel("Log Time")
plt.ylabel("Log Error")
plt.title("Log of Learning Error")
plt.show()

In [ ]:
std = np.std(MC_regret, axis=0)
ci = 1.96 * std/np.sqrt(len(agent.ts))
y = np.mean(MC_regret, axis=0)

fig, ax = plt.subplots()
ax.plot(agent.ts, y)
ax.fill_between(agent.ts, (y-ci), (y+ci), color='b', alpha=.1)
plt.xlabel("Time")
plt.ylabel("Regret")
plt.title("Algorithm Regret")
plt.show()

In [ ]:
y_axis = (y)[10:]
y_axis_upper = (y+ci)[10:]
x_axis = agent.ts[10:]

# Define the quadratic model
def log_square_model(x, c, m):
    return c * (np.log(x)**2) + m

# Perform the regression
params, covariance = curve_fit(log_square_model, x_axis, y_axis)
c_estimated = params[0]
m_estimated = params[1]

# Perform the regression
params, covariance = curve_fit(log_square_model, x_axis, y_axis_upper)
c_estimated_upper = params[0]
m_estimated_upper = params[1]

In [ ]:
fig, ax = plt.subplots()
ax.plot(agent.ts, y, label='(expected) regret')
ax.fill_between(agent.ts, (y-ci), (y+ci), color='b', alpha=.1, label='$95\%$ CI of (expected) regret')
x_fit = np.linspace(10, T, 1001)
y_fit = log_square_model(x_fit, c_estimated, m_estimated)
y_fit_upper = log_square_model(x_fit, c_estimated_upper, m_estimated_upper)
ax.plot(x_fit, y_fit, color='mediumslateblue',linestyle=':', label=f'Fitted curve ${np.round(c_estimated, 3)} \ln(T)^2 + {np.round(m_estimated,4)}$')
ax.plot(x_fit, y_fit_upper, color='mediumslateblue',linestyle='dashdot', label=f'Fitted curve ${np.round(c_estimated_upper, 4)} \ln(T)^2 + {np.round(m_estimated_upper,4)}$')
plt.xlabel("Time")
plt.ylabel("Regret")
plt.title("Algorithm Regret")
plt.legend()
plt.show()